In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
# Measuring mass of disc D1
M = np.array([...]) # Mass measured in grams

mass_mean = np.mean(M)

mass_stdev = np.std(M, ddof=1)

mass_stderr = mass_stdev / np.sqrt(len(M))

print(f"Mean Mass: {mass_mean:.4f} g")
print(f"Standard Deviation: {mass_stdev:.4f} g")
print(f"Standard Error: {mass_stderr:.4f} g")

In [ ]:
# Measuring diameter of specimen disc d
D = np.array([...]) # In cm

dia_mean = np.mean(D)

dia_stdev = np.std(D, ddof=1)

dia_stderr = dia_stdev / np.sqrt(len(D))

print(f"Mean Diameter: {dia_mean:.4f} cm")
print(f"Standard Deviation: {dia_stdev:.4f} cm")
print(f"Standard Error: {dia_stderr:.4f} cm")

In [ ]:
# Measuring thickness of specimen disc d
L = np.array([...]) # in mm

thick_mean = np.mean(L)

thick_stdev = np.std(L, ddof=1)

thick_stderr = thick_stdev / np.sqrt(len(L))

print(f"Mean Thickness: {thick_mean:.4f} mm")
print(f"Standard Deviation: {thick_stdev:.4f} mm")
print(f"Standard Error: {thick_stderr:.4f} mm")

In [ ]:
# Initial Temperature Difference T'
T2_init = ...
T1_init = ...
T_prime = T2_init - T1_init
print(f"Initial Temperature Difference (T'): {T_prime} deg C")

In [ ]:
T1_steady = ...
T2_steady = ...

delta_T_val = T2_steady - T1_steady - T_prime
print(f"Steady State Temperature Difference: {delta_T_val} deg C")

In [ ]:
# Cooling Data

# Time in seconds
time_cooling = np.array([...])

# Temperature in degree Celsius
temp_cooling = np.array([...])

In [ ]:
# INPUTS and CONSTANTS
Tamb = ... # Ambient Temperature (deg C)

mass_val = np.mean(M) # Mean Mass (g)
thick_val = np.mean(L) # Mean Thickness (mm)
dia_val = np.mean(D) # Mean Diameter (cm)

# Least Counts
LC_mass = ... # Balance LC (g)
LC_screw = ... # Screw Gauge LC (mm)
LC_vernier = ... # Vernier LC (cm)
LC_temp = ... # Thermometer LC (deg C)

# Other Constants
sp_heat_s  = 380.0 # J/kg/K (Brass)

# Convert to SI Units
m_kg = mass_val / 1000.0
l_m = thick_val / 1000.0
d_m = dia_val / 100.0

delta_m = LC_mass
delta_l = LC_screw
delta_d = LC_vernier
delta_T = 2.0 * LC_temp 

# Area & its error
radius_m = d_m / 2.0
Area = np.pi * radius_m**2
fractional_area = 2 * (delta_d / 100.0) / d_m

# Curve fitting
def cooling_model(t, A, k, T_env):
    return A * np.exp(-k * t) + T_env

t_range = max(time_cooling) - min(time_cooling)
T_drop = max(temp_cooling) - min(temp_cooling)

guess_T_env = Tamb  # Ambient Temperature
guess_A = temp_cooling[0] - guess_T_env 
guess_k = 1.0 / (t_range / 2.0)

p0_auto = [guess_A, guess_k, guess_T_env]

try:
    popt, pcov = curve_fit(cooling_model, time_cooling, temp_cooling, p0=p0_auto)
except RuntimeError:
    print("Failed fitting, you are cooked")
    popt = p0_auto

A_fit, k_fit, T_env_fit = popt

# Calculate Slope (dT/dt) at T1_steady:

# Find time 't' when Temp = T1_steady using inverse model
t_at_T1 = (-1/k_fit) * np.log((T1_steady - T_env_fit) / A_fit)

slope_val = abs(-A_fit * k_fit * np.exp(-k_fit * t_at_T1))

perr = np.sqrt(np.diag(pcov))
rel_err_A = perr[0] / A_fit
rel_err_k = perr[1] / k_fit
std_err_slope = slope_val * np.sqrt(rel_err_A**2 + rel_err_k**2)

# Calculate Thermal Conductivity (K)
k_value = (m_kg * sp_heat_s * slope_val * l_m) / (Area * delta_T_val)

# Calculate fractional errors for quadrature
frac_m = (delta_m / 1000.0) / m_kg
frac_l = (delta_l / 1000.0) / l_m
frac_slope = std_err_slope / slope_val
frac_T = delta_T / delta_T_val 

# Total Fractional Error
total_frac_error = np.sqrt(frac_m**2 + frac_l**2 + fractional_area**2 + frac_slope**2 + frac_T**2)
delta_k = k_value * total_frac_error
percentage_error = total_frac_error * 100

print(f"Slope (dT/dt) at {T1_steady}: {slope_val:.4f} C/s")
print(f"Uncertainty in slope: {std_err_slope:.4f} C/s")
print(f"Calculated k: {k_value:.4f} W/(mC)")
print(f"Uncertainty in k: {delta_k:.4f} W/(mC)")
print(f"Percentage Error: {percentage_error:.2f}%")

plt.figure(figsize=(10, 6))

plt.errorbar(time_cooling, temp_cooling, yerr=LC_temp, fmt='o', color='red', label='Experimental Data')

# Plot Fit
x_fit = np.linspace(min(time_cooling), max(time_cooling), 100)
y_fit = cooling_model(x_fit, *popt)
plt.plot(x_fit, y_fit, 'b-', label=f'Exp Fit: $T={A_fit:.1f}e^{{-{k_fit:.3f}t}} + {T_env_fit:.1f}$')

# Plot Tangent
tangent = -slope_val * (x_fit - t_at_T1) + T1_steady
mask = np.abs(x_fit - t_at_T1) < 60
plt.plot(x_fit[mask], tangent[mask], 'g--', linewidth=2, label=f'Slope at $T_1$: {slope_val:.4f}')

plt.axvline(t_at_T1, color='gray', linestyle=':')
plt.axhline(T1_steady, color='gray', linestyle=':')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Temperature (deg C)')
plt.title('Cooling Curve Analysis (Newton\'s Law)')
plt.grid(True, alpha=0.5)
plt.show()